<a href="https://colab.research.google.com/github/RayBasilio123/IC_2022/blob/main/hyperForest_Manaus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import f1_score
from datetime import datetime
from matplotlib import pyplot
from sklearn.ensemble import RandomForestRegressor
from math import nan
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV


In [ ]:
def get_x2(df,lista,lags,Eto,lags_eto):
  lags=lags
  lista_aux=[]
  lista_aux2=[]
  max_lag=0;
  data = pd.DataFrame()
  eta_nois = pd.DataFrame()
  eta_nois['Data']=df['Data']
  for coluna in lista:
    data[coluna] = df[coluna]       

  for i in range(len(lags)):
    for j in range(len(lags[i])):
      lista_aux = data.iloc[:,i].tolist()
      for displacement in range((lags[i][j])):
        if max_lag<(lags[i][j]):
          max_lag=(lags[i][j])
        del lista_aux[len(lista_aux)-1]
        lista_aux.insert(0,nan)
      eta_nois[((data.iloc[:,i]).name)+("_t-")+str(lags[i][j])]=(lista_aux)  
  
  for i in range(len(lags_eto)):
    lista_aux2=df[Eto].iloc[:,0].tolist()
    for displacement in range((lags_eto[i])):
      if max_lag<(lags_eto[i]):
        max_lag=(lags_eto[i])
      del lista_aux2[len(lista_aux2)-1]
      lista_aux2.insert(0,nan)
    eta_nois[((df[Eto].iloc[:,0]).name)+("_t-")+str(lags_eto[i])]=lista_aux2        
  return eta_nois,max_lag,lista,lags,Eto,lags_eto;

In [ ]:
def get_x30(df,lista, Eto):
  ix = []
  idx =  [i for i in np.arange(1, 31)]
  for i in range(len(lista)):
    ix.append(idx)

  resultado = get_x2(df, lista, ix, Eto, idx)
  return resultado

In [ ]:
def column_Filter(df,dias):
  list_column = []
  # pegando colunas do data frame
  for column in (df.columns):
    if column != 'Data':
      list_column.append(column)
  # prosseguindo com a filtração
  b=[]
  lags=[]
  aux3=[]
  lista=[]
  Eto=[]
  lags_eto=[]
 
  # filtra e  remove colunas 
  for i in range(len(list_column)):
    j=list_column[i].split("_t-")
    if (int(j[1])>=dias):
      b.append(j)
  # print("Lista_Filtrada",b)

  #localiza as colunas  e faz uma lista se a coluna for Eto cria uma lista só pra ela
  for k in range(len(b)):
    if b[k][0] not in lista and b[k][0]!="Eto":
        lista.append(b[k][0])
    elif b[k][0] not in Eto and b[k][0]=="Eto":
        Eto.append(b[k][0])

  # print("lista",lista)
  # print("Eto",Eto)
 
  # localiza as colunas e cria uma lista de lags para cada
  for i in range(len(lista)):
    aux=[]
    for k in range(len(b)):
      if (lista[i]==b[k][0])  :
          aux.append(int(b[k][1]))
    lags.append(aux)
    del(aux)

  # print("lags",lags)
           
  # print("Eto",Eto)
  
  
  if Eto!=[]:
    for k in range(len(b)):
      if b[k][0]==Eto[0]:
        lags_eto.append(int(b[k][1]))
  # print("lags_eto",lags_eto)
 
  aux3.append(lista)
  aux3.append(lags)
  aux3.append(Eto)
  aux3.append(lags_eto)
 

  return aux3









In [ ]:
def resource_ranking(df,lista_filtrada3,variavel_alvo) :
  
  # resource_ranking(df_la,lista_filtrada2,"temp_min")  
  #Monta a tabela com os dados filtrados 
  
  tab=get_x2(df,lista_filtrada3[0],lista_filtrada3[1],lista_filtrada3[2],lista_filtrada3[3])
 
  # dff=tab[0].drop("Data",axis=1)
  
  # train_size = int(((len(tab[0]))-tab[1]) )
  # dff=dff.iloc[tab[1]:train_size,:]

  # array1 = dff.values

  # df= df.drop("Data",axis=1)
  # df=df.iloc[tab[1]:train_size,:]
  # array2 = df[variavel_alvo]
  selecao_treino = (tab[0]['Data'] >= '2014') & (tab[0]['Data'] <= '2018-12-31')
  
  dff=tab[0][selecao_treino].drop("Data",axis=1)

  train_size = int(((len(tab[0][selecao_treino]))-tab[1]) )
  dff=dff.iloc[tab[1]:train_size,:]
  array1 = dff.values

  df= df[selecao_treino].drop("Data",axis=1)
  df=df.iloc[tab[1]:train_size,:]
  array2 = df[variavel_alvo]

 

  X =array1[:,0:len(tab[0].columns)]
  Y = array2
  # feature extraction
  test = SelectKBest(score_func=f_regression, k=4)
  fit = test.fit(X, Y)
  # summarize scores
  set_printoptions(precision=3)
 

    
  # print("Selecao_univariada",fit.scores_)
  f=fit.scores_
  f_ord = sorted(f,reverse=True)
  # print("Selecao_univariada_ordenada",f_ord)

  ll=[]
  for y in range(len(f_ord)):
    for i in range(len(f)):
      if (f_ord[y]==f[i]):
        ll.append(i)
        # print("f[i]",f[i])
        # print("f_ord[y]",f_ord[y])
  leg_seq=[]
  for i in range(len(ll)):
    leg_seq.append(dff.columns[ll[i]])    

  # print("Colunas_selecionadas [0]- Selecao_univariada",leg_seq)
 
  model = ExtraTreesRegressor(n_estimators=10,random_state=42)
  model.fit(X,Y)
  g=model.feature_importances_
  g_ord=sorted(model.feature_importances_,reverse=True)
  # print("Importancia",g)
  # print("Importancia_ordenada",g_ord)
  jj=[]
  for y in range(len(g_ord)):
    for i in range(len(g)):
      if (g_ord[y]==g[i]):
        jj.append(i)
  leg_seq2=[]
  for i in range(len(ll)):
    leg_seq2.append(dff.columns[jj[i]])    
  # print("Colunas_selecionadas [1]- Importância do recurso",leg_seq2)
  return leg_seq,leg_seq2

In [ ]:


def list_Format(melhores_recursos,q,dias):
  b=[]
  lags=[]
  aux3=[]
  lista=[]
  Eto=[]
  lags_eto=[]
  q = len(melhores_recursos) if q==0 else q
  # filtra e  remove colunas 
  for i in range(q):
    j=melhores_recursos[i].split("_t-")
    if (int(j[1])>=dias):
      b.append(j)
  print("lista_selecionada",b)

  #localiza as colunas  e faz uma lista se a coluna for Eto cria uma lista só pra ela
  for k in range(len(b)):
    if b[k][0] not in lista and b[k][0]!="Eto":
        lista.append(b[k][0])
    elif b[k][0] not in Eto and b[k][0]=="Eto":
        Eto.append(b[k][0])

  print("lista",lista)
  print("Eto",Eto)
 
  # localiza as colunas e cria uma lista de lags para cada
  for i in range(len(lista)):
    aux=[]
    for k in range(len(b)):
      if (lista[i]==b[k][0])  :
          aux.append(int(b[k][1]))
    lags.append(aux)
    del(aux)

  print("lags",lags)
           
  # print("Eto",Eto)
  
  
  if Eto!=[]:
    for k in range(len(b)):
      if b[k][0]==Eto[0]:
        lags_eto.append(int(b[k][1]))
  print("lags_eto",lags_eto)
 
  aux3.append(lista)
  aux3.append(lags)
  aux3.append(Eto)
  aux3.append(lags_eto)
 

  return aux3

In [ ]:
def train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo,data_Itest= '2019-01-01',data_Ftest= '2020-12-31',data_Itreino='2014-01-01',data_Ftreino='2018-12-31'):
  tabela = get_x2(df, lista,lags,Eto,lags_eto)
  selecao_treino = (tabela[0]['Data'] >= data_Itreino) & (tabela[0]['Data'] <= data_Ftreino)
  selecao_teste = (tabela[0]['Data'] >= data_Itest) & (tabela[0]['Data'] <= data_Ftest)

  x1_train = tabela[0][selecao_treino].drop("Data", axis=1)
  x1_test = tabela[0][selecao_teste].drop("Data", axis=1)
  
  y1_train = df[variavel_Alvo][selecao_treino]
  y1_test = df[variavel_Alvo][selecao_teste]

  x1_train = x1_train[tabela[1]:]
  y1_train = y1_train[tabela[1]:]

  
  print("***************************************")
  print(x1_train.shape,"x1_train")
  #[6569 rows x 10 columns] x1_train
  print("*****************++*********************")
  print(x1_test.shape,"x1_test")
  print("******************---********************")
  print(y1_train.shape,"y1_train")
  print("****************1234*********************")
  print(y1_test.shape,"y1_test")
  print("*****************asddfg******************")
 
  
  return x1_train, x1_test,y1_train, y1_test

In [ ]:
def gridSearchF(df, lista,lags,Eto,lags_eto,variavel_Alvo):
  x1_train, x1_test,y1_train, y1_test = train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo)
   # Create the parameter grid based on the results of random search 
  param_grid = {
      'bootstrap': [True],
      'max_depth': [10,80, 90, 100, 110],
      'max_features': [2, 4,5],
      'min_samples_leaf': [3, 4, 5],
      'min_samples_split': [2,8, 10, 12],
      'n_estimators': [100,300, 1000]
  }

  # Create a base model
  rf = RandomForestRegressor(random_state = 42)

  # Instantiate the grid search model
  grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                            cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

  print(" -----------------grid_search.fit--------------")
 
  start_time=timer(None)

  grid_search.fit(x1_train, y1_train);

  timer(start_time)  
  
    
  return grid_search.best_params_

def gridSearchFs(df,parametros,variavel_Alvo):

  lista_colunas=["gridSearch"]
  tb = pd.DataFrame(columns=lista_colunas)
  
  print("gridSearchFs")
  print('---------------------------------------------------------------------------------')
  
  for x in range(len(parametros)):
    a=gridSearchF(df,parametros[x][0],parametros[x][1],parametros[x][2],parametros[x][3],variavel_Alvo)
    
    print(x,"Best_hyperparameter_Arvore", a)
   
  print('---------------------------------------------------------------------------------')
  

  return a




In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time=datetime.now()
        return start_time
    elif start_time:
        thour,temp_sec=divmod((datetime.now()-start_time).total_seconds(),3600)
        tmin,tsec=divmod(temp_sec,60)

In [ ]:
url=('https://raw.githubusercontent.com/RayBasilio123/database/main/INMET/CSV/J_Database/MANAUS_J_2014_2020.csv')
df = pd.read_csv(url, sep=",", encoding = "ISO-8859-1")
df=df.drop(columns=['Unnamed: 0'])

In [ ]:
df

,Data,vento,radiacao,temp_max,temp_min,umi_max,umi_min,umi_rel,press_atm,temp_media,Eto,J
0,2014-01-01,1.343195,1.599215,33.9,26.0,77.0,38,58.541667,1002.579167,29.504167,4.997605,1
1,2014-01-02,1.408641,0.920206,34.5,25.3,75.0,38,56.708333,1001.154167,29.700000,5.359660,2
2,2014-01-03,1.149975,0.872915,33.5,25.5,80.0,43,60.916667,1001.616667,29.462500,4.187175,3
3,2014-01-04,1.368127,0.855046,33.6,26.5,79.0,43,63.416667,1002.937500,29.177083,4.487894,4
4,2014-01-05,1.159324,0.655731,31.4,23.5,97.0,54,71.250000,1003.525000,27.766667,3.100133,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2552,2020-12-27,1.564464,0.759292,31.6,24.1,91.0,56,76.479167,1001.247917,27.081250,2.990388,362
2553,2020-12-28,1.405525,0.650427,32.1,25.5,89.0,55,76.812500,1002.956250,27.636458,2.835460,363
2554,2020-12-29,1.215420,0.473719,31.5,24.9,93.0,56,81.229167,1003.039583,26.645833,2.011841,364
2555,2020-12-30,1.131276,0.735296,31.4,22.3,96.0,52,79.541667,1004.547917,25.605208,2.204808,365


In [ ]:
atributeP= [ 'vento'	,'radiacao',	'temp_max',	'temp_min',	'umi_max',	'umi_min',	'umi_rel',	'press_atm',	'temp_media','J']
resultadoP=get_x30(df,atributeP,["Eto"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
for x in [1,3,7,10]:
 lista_Filtrada=column_Filter(resultadoP[0],x)
 ranking_data = resource_ranking(df,lista_Filtrada,'Eto')
 lista_Formatada1=list_Format(ranking_data[0],10,0)
 lista_Formatada2=list_Format(ranking_data[1],10,0)
 print('\n -----------Para----' + str(x) + '----dias --------\n')
 print("lista_Formatada_1 -------> ",lista_Formatada1)
 print("lista_Formatada_2 -------> ",lista_Formatada2)
 print("lista_Formatada_3 -------> ",lista_Filtrada)
 print('\n ---------------------------------------------\n')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


lista_selecionada [['Eto', '1'], ['umi_rel', '1'], ['umi_min', '1'], ['temp_media', '1'], ['temp_max', '1'], ['Eto', '2'], ['radiacao', '1'], ['umi_max', '1'], ['temp_min', '1'], ['umi_rel', '2']]
lista ['umi_rel', 'umi_min', 'temp_media', 'temp_max', 'radiacao', 'umi_max', 'temp_min']
Eto ['Eto']
lags [[1, 2], [1], [1], [1], [1], [1], [1]]
lags_eto [1, 2]
lista_selecionada [['Eto', '1'], ['umi_min', '1'], ['umi_rel', '1'], ['temp_max', '1'], ['temp_media', '2'], ['radiacao', '1'], ['temp_media', '1'], ['vento', '1'], ['umi_max', '1'], ['umi_rel', '16']]
lista ['umi_min', 'umi_rel', 'temp_max', 'temp_media', 'radiacao', 'vento', 'umi_max']
Eto ['Eto']
lags [[1], [1, 16], [1], [2, 1], [1], [1], [1]]
lags_eto [1]

 -----------Para----1----dias --------

lista_Formatada_1 ------->  [['umi_rel', 'umi_min', 'temp_media', 'temp_max', 'radiacao', 'umi_max', 'temp_min'], [[1, 2], [1], [1], [1], [1], [1], [1]], ['Eto'], [1, 2]]
lista_Formatada_2 ------->  [['umi_min', 'umi_rel', 'temp_max', 'te

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


lista_selecionada [['umi_rel', '3'], ['umi_rel', '13'], ['Eto', '3'], ['umi_rel', '9'], ['umi_rel', '8'], ['umi_rel', '4'], ['umi_rel', '12'], ['umi_rel', '14'], ['umi_rel', '7'], ['umi_rel', '10']]
lista ['umi_rel']
Eto ['Eto']
lags [[3, 13, 9, 8, 4, 12, 14, 7, 10]]
lags_eto [3]
lista_selecionada [['J', '4'], ['umi_min', '4'], ['J', '5'], ['J', '19'], ['J', '15'], ['J', '6'], ['J', '13'], ['J', '7'], ['J', '21'], ['Eto', '13']]
lista ['J', 'umi_min']
Eto ['Eto']
lags [[4, 5, 19, 15, 6, 13, 7, 21], [4]]
lags_eto [13]

 -----------Para----3----dias --------

lista_Formatada_1 ------->  [['umi_rel'], [[3, 13, 9, 8, 4, 12, 14, 7, 10]], ['Eto'], [3]]
lista_Formatada_2 ------->  [['J', 'umi_min'], [[4, 5, 19, 15, 6, 13, 7, 21], [4]], ['Eto'], [13]]
lista_Formatada_3 ------->  [['vento', 'radiacao', 'temp_max', 'temp_min', 'umi_max', 'umi_min', 'umi_rel', 'press_atm', 'temp_media', 'J'], [[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


lista_selecionada [['umi_rel', '13'], ['umi_rel', '9'], ['umi_rel', '8'], ['umi_rel', '12'], ['umi_rel', '14'], ['umi_rel', '7'], ['umi_rel', '10'], ['Eto', '13'], ['Eto', '8'], ['umi_rel', '11']]
lista ['umi_rel']
Eto ['Eto']
lags [[13, 9, 8, 12, 14, 7, 10, 11]]
lags_eto [13, 8]
lista_selecionada [['temp_max', '8'], ['J', '13'], ['umi_rel', '9'], ['umi_min', '16'], ['J', '14'], ['J', '16'], ['umi_rel', '13'], ['J', '9'], ['J', '27'], ['J', '10']]
lista ['temp_max', 'J', 'umi_rel', 'umi_min']
Eto []
lags [[8], [13, 14, 16, 9, 27, 10], [9, 13], [16]]
lags_eto []

 -----------Para----7----dias --------

lista_Formatada_1 ------->  [['umi_rel'], [[13, 9, 8, 12, 14, 7, 10, 11]], ['Eto'], [13, 8]]
lista_Formatada_2 ------->  [['temp_max', 'J', 'umi_rel', 'umi_min'], [[8], [13, 14, 16, 9, 27, 10], [9, 13], [16]], [], []]
lista_Formatada_3 ------->  [['vento', 'radiacao', 'temp_max', 'temp_min', 'umi_max', 'umi_min', 'umi_rel', 'press_atm', 'temp_media', 'J'], [[7, 8, 9, 10, 11, 12, 13, 14, 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


lista_selecionada [['umi_rel', '13'], ['umi_rel', '12'], ['umi_rel', '14'], ['umi_rel', '10'], ['Eto', '13'], ['umi_rel', '11'], ['umi_rel', '15'], ['umi_rel', '16'], ['Eto', '14'], ['umi_rel', '17']]
lista ['umi_rel']
Eto ['Eto']
lags [[13, 12, 14, 10, 11, 15, 16, 17]]
lags_eto [13, 14]
lista_selecionada [['J', '21'], ['J', '14'], ['J', '30'], ['umi_min', '13'], ['J', '12'], ['J', '11'], ['J', '23'], ['J', '24'], ['J', '16'], ['J', '15']]
lista ['J', 'umi_min']
Eto []
lags [[21, 14, 30, 12, 11, 23, 24, 16, 15], [13]]
lags_eto []

 -----------Para----10----dias --------

lista_Formatada_1 ------->  [['umi_rel'], [[13, 12, 14, 10, 11, 15, 16, 17]], ['Eto'], [13, 14]]
lista_Formatada_2 ------->  [['J', 'umi_min'], [[21, 14, 30, 12, 11, 23, 24, 16, 15], [13]], [], []]
lista_Formatada_3 ------->  [['vento', 'radiacao', 'temp_max', 'temp_min', 'umi_max', 'umi_min', 'umi_rel', 'press_atm', 'temp_media', 'J'], [[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3

# Para  1 dia - Grid Forest


In [ ]:
lista_Formatada_1=  [['umi_rel', 'umi_min', 'temp_media', 'temp_max', 'radiacao', 'umi_max', 'temp_min'], [[1, 2], [1], [1], [1], [1], [1], [1]], ['Eto'], [1, 2]]
lista_Formatada_2=  [['umi_min', 'umi_rel', 'temp_max', 'temp_media', 'radiacao', 'vento', 'umi_max'], [[1], [1, 16], [1], [2, 1], [1], [1], [1]], ['Eto'], [1]]

In [ ]:
parametrosSearchA=gridSearchFs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")

gridSearchFs
---------------------------------------------------------------------------------
***************************************
(1824, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1824,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
0 Best_hyperparameter_Arvore {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 300}
***************************************
(1810, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1810,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candi

#Para  3 dias - Grid Forest

In [ ]:
lista_Formatada_1=  [['umi_rel'], [[3, 13, 9, 8, 4, 12, 14, 7, 10]], ['Eto'], [3]]
lista_Formatada_2=  [['J', 'umi_min'], [[4, 5, 19, 15, 6, 13, 7, 21], [4]], ['Eto'], [13]]

In [ ]:
parametrosSearchA=gridSearchFs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")


gridSearchFs
---------------------------------------------------------------------------------
***************************************
(1812, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1812,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
0 Best_hyperparameter_Arvore {'bootstrap': True, 'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 4, 'min_samples_split': 12, 'n_estimators': 100}
***************************************
(1805, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1805,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candi

# Para 7 dias Grid Forest

In [ ]:
lista_Formatada_1=  [['umi_rel'], [[13, 9, 8, 12, 14, 7, 10, 11]], ['Eto'], [13, 8]]
lista_Formatada_2=  [['temp_max', 'J', 'umi_rel', 'umi_min'], [[8], [13, 14, 16, 9, 27, 10], [9, 13], [16]], [], []]

In [ ]:
parametrosSearchA=gridSearchFs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")


gridSearchFs
---------------------------------------------------------------------------------
***************************************
(1812, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1812,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
0 Best_hyperparameter_Arvore {'bootstrap': True, 'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 5, 'min_samples_split': 12, 'n_estimators': 300}
***************************************
(1799, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1799,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candi

# Para 10 dias - Grid Forest

In [ ]:
lista_Formatada_1=  [['umi_rel'], [[13, 12, 14, 10, 11, 15, 16, 17]], ['Eto'], [13, 14]]
lista_Formatada_2=  [['J', 'umi_min'], [[21, 14, 30, 12, 11, 23, 24, 16, 15], [13]], [], []]

In [ ]:
parametrosSearchA=gridSearchFs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")


gridSearchFs
---------------------------------------------------------------------------------
***************************************
(1809, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1809,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
0 Best_hyperparameter_Arvore {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 5, 'min_samples_split': 12, 'n_estimators': 300}
***************************************
(1796, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1796,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candi